<center>
  <h1>Playing with Python Bytecode</h1><br><br>
  <h2>A Metatalk About Metaprogramming</h2><br><br>
  <h3><a href="https://github.com/ssanderson/pycon-2016">https://github.com/ssanderson/pycon-2016</a>
  </h3><br>
</center>

In [8]:
def add(a, b):
    return a + b
add(1, 2)

3

In [16]:
# add.__code__
# add.__code__.co_<TAB>
# add.__code__.co_argcount
# add.__code__.co_varnames
# add.__code__.co_lnotab
# add.__code__.co_flags (that makes sense)
add.__code__.co_code

b'|\x00\x00|\x01\x00\x17S'

In [54]:
# import this
# import dis

print(list(add.__code__.co_code))
print()
dis.dis(add)

[124, 0, 0, 124, 1, 0, 23, 83]

  2           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 RETURN_VALUE


In [29]:
def add_with_assign(a, b):
    x = a + b
    return x
dis.dis(add_with_assign)

  2           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 STORE_FAST               2 (x)

  3          10 LOAD_FAST                2 (x)
             13 RETURN_VALUE


In [43]:
def abs(x):
    if x > 0:
        return x
    else:
        return -x
dis.dis(abs)

  2           0 LOAD_FAST                0 (x)
              3 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
              9 POP_JUMP_IF_FALSE       16

  3          12 LOAD_FAST                0 (x)
             15 RETURN_VALUE

  5     >>   16 LOAD_FAST                0 (x)
             19 UNARY_NEGATIVE
             20 RETURN_VALUE
             21 LOAD_CONST               0 (None)
             24 RETURN_VALUE


In [44]:
# Scott: Well that seems wasteful. I want to make my own!
# Joe:   Hold on there killer.  Before we dive into making our own abs(), why don't
#        you start with something a bit simpler first.
# Scott: Alright, I'll write addone.
def addone(x):
    return x + 1

In [55]:
# Scott: Okay, so no,w I guess I need to make a code object...how do I do that?
# Joe:   Well, you could just call type(addone.__code__), but there's a 'types' module that 
#        exports all the builtin types.
from types import CodeType
CodeType?

In [ ]:
# def addone(x):
#     return x + 1
my_code = CodeType(1, # argcount
                   0, # kwonlyargcount
                   1, # nlocals
                   2, # stacksize
                    

Flags Slide

In [57]:
# def addone(x):
#     return x + 1
my_code = CodeType(1,             # argcount
                   0,             # kwonlyargcount
                   1,             # nlocals
                   2,             # stacksize
                   (0x0001 | 0x0002 | 0x0040),
# Resume Typing
                   bytes([124, 0, 0, 100, 0, 0, 23, 83]),
                   (1,),          # constants
                   (),            # names
                   ('x',),        # varnames
                   '<string>',    # filename
                   'addone',      # name
                   1,             # firstlineno
                   bytes([0, 1])) # lnotab

In [58]:
dis.dis(my_code)

  2           0 LOAD_FAST                0 (x)
              3 LOAD_CONST               0 (1)
              6 BINARY_ADD
              7 RETURN_VALUE


In [60]:
my_code(1)

TypeError: 'code' object is not callable

In [62]:
from types import FunctionType
FunctionType?

my_addone = FunctionType(my_code, {})

In [63]:
my_addone(1)

2